# Computing fourth-order shear statistics on a mock shape catalog

In this notebooks we compute the fourth-order shear correlation functions on a realistic shape catalog and compare it with the gaussian prediction

In [1]:
from astropy.table import Table
import numpy as np
from matplotlib import pyplot as plt

import orpheus

## Obtaining a realistic mock shape catalog

We will run this notebook using a mocks shape catalog from the SLICS ensemble. First, let us download this catalog and read in its contents.

**Note:** If you want to run the notebook yourself, you need to update the `savepath_SLICS` variable.

In [2]:
catname = "GalCatalog_LOS1.fits"
path_to_SLICS = "http://cuillin.roe.ac.uk/~jharno/SLICS/MockProducts/KiDS450/" + catname
savepath_SLICS = "/vol/euclidraid4/data/lporth/HigherOrderLensing/Mocks/SLICS_KiDS450/"

In [3]:
!wget {path_to_SLICS} -P {savepath_SLICS}

--2025-09-10 00:48:32--  http://cuillin.roe.ac.uk/~jharno/SLICS/MockProducts/KiDS450/GalCatalog_LOS1.fits
Resolving cuillin.roe.ac.uk (cuillin.roe.ac.uk)... 129.215.175.32
Connecting to cuillin.roe.ac.uk (cuillin.roe.ac.uk)|129.215.175.32|:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 196545600 (187M) [image/fits]
Saving to: ‘/vol/euclidraid4/data/lporth/HigherOrderLensing/Mocks/SLICS_KiDS450/GalCatalog_LOS1.fits.2’

GalCatalog_LOS1.fit 100%[===================>] 187,44M  29,2MB/s    in 5,3s    

2025-09-10 00:48:38 (35,3 MB/s) - ‘/vol/euclidraid4/data/lporth/HigherOrderLensing/Mocks/SLICS_KiDS450/GalCatalog_LOS1.fits.2’ saved [196545600/196545600]



In [3]:
slicscat = Table.read(savepath_SLICS + catname)
print(slicscat.keys())

['x_arcmin', 'y_arcmin', 'z_spectroscopic', 'z_photometric', 'shear1', 'shear2', 'eps_obs1', 'eps_obs2']


In [8]:
fdownsample = 10

In [9]:
shapecat = orpheus.SpinTracerCatalog(spin=2,
                                     pos1=slicscat["x_arcmin"][::fdownsample],
                                     pos2=slicscat["y_arcmin"][::fdownsample],
                                     tracer_1=slicscat["shear1"][::fdownsample],
                                     tracer_2=slicscat["shear2"][::fdownsample])

In [10]:
print("Number of galaxies:%i --> effective nbar: %.3f/arcmin^2 on %.2f deg^2"%(
    shapecat.ngal, shapecat.ngal/(shapecat.len1*shapecat.len2), shapecat.len1*shapecat.len2/3600.))

Number of galaxies:307081 --> effective nbar: 0.853/arcmin^2 on 100.00 deg^2


In [11]:
min_sep = 0.125
max_sep = 128.
binsize = 0.2
nthreads = 48
nmax=10
mapradii = np.geomspace(2,32,17)

In [12]:
fourpcf = orpheus.GGGGCorrelation_NoTomo(min_sep=min_sep, max_sep=max_sep,binsize=binsize,nmaxs=nmax,method='Tree',
                                         nthreads=nthreads,verbosity=2)

In [13]:
%%time
map4 = fourpcf.process(shapecat, lowmem=True, statistics=['4pcf_multipole','Map4'],mapradii=mapradii)

Using batchsize of 161 for radial bins
Doing region 0/86996 for thetabatch 0/48
Computed Gn for first gal in region 0/86996 for thetabatch 0/48 in 0.0072 seconds
nindex 0: n2=-21 n3=10
nindex 1: n2=-21 n3=11
nindex 2: n2=-20 n3=9
nindex 3: n2=-20 n3=10
nindex 4: n2=-20 n3=11
nindex 5: n2=-19 n3=8
nindex 6: n2=-19 n3=9
nindex 7: n2=-19 n3=10
nindex 8: n2=-19 n3=11
nindex 9: n2=-18 n3=7
nindex 10: n2=-18 n3=8
nindex 11: n2=-18 n3=9
nindex 12: n2=-18 n3=10
nindex 13: n2=-18 n3=11
nindex 14: n2=-17 n3=6
nindex 15: n2=-17 n3=7
nindex 16: n2=-17 n3=8
nindex 17: n2=-17 n3=9
nindex 18: n2=-17 n3=10
nindex 19: n2=-17 n3=11
nindex 20: n2=-16 n3=5
nindex 21: n2=-16 n3=6
nindex 22: n2=-16 n3=7
nindex 23: n2=-16 n3=8
nindex 24: n2=-16 n3=9
nindex 25: n2=-16 n3=10
nindex 26: n2=-16 n3=11
nindex 27: n2=-15 n3=4
nindex 28: n2=-15 n3=5
nindex 29: n2=-15 n3=6
nindex 30: n2=-15 n3=7
nindex 31: n2=-15 n3=8
nindex 32: n2=-15 n3=9
nindex 33: n2=-15 n3=10
nindex 34: n2=-15 n3=11
nindex 35: n2=-14 n3=3
nindex

In [17]:
fourpcf.npcf_multipoles_norm.shape

(21, 21, 1, 35, 35, 35)